# 06_Base_sumarizada

## 01_import_bibliotecas

In [ ]:
# === Geração da base consolidada para análise regional por estado + cluster ===

import pandas as pd
import numpy as np
from datetime import datetime
import joblib


## 02_Carga_de_dados

In [ ]:

# === 1. Carregar base original ===
df_2 = pd.read_parquet("01_candidatos.parquet")

## 03_Tratamento

In [ ]:
# === 2. Calcular idade com base em data_nascimento ===
hoje = pd.to_datetime("today")
df_2["data_nascimento"] = pd.to_datetime(df_2["data_nascimento"], errors="coerce")
df_2["idade"] = df_2["data_nascimento"].apply(lambda x: hoje.year - x.year if pd.notnull(x) else np.nan)


In [ ]:
# === 3. Carregar modelo e colunas usadas ===
modelo = joblib.load("03_modelos/modelo_kmeans.joblib")
colunas_modelo = joblib.load("03_modelos/colunas_usadas.joblib")

In [ ]:
# === 4. Preparar features de entrada ===
df_model = df_2.copy()

# Preencher flags e sentinelas
for col in colunas_modelo:
    if col.startswith("tem_"):
        original = col.replace("tem_", "")
        df_model[col] = df_model[original].notnull().astype(int)
        df_model[original] = df_model[original].fillna(-9999)

# Garantir todas as colunas
for col in colunas_modelo:
    if col not in df_model.columns:
        df_model[col] = 0

In [ ]:
# === 5. Prever cluster para cada candidato ===
X_input = df_model[colunas_modelo]
df_2["cluster"] = modelo.predict(X_input)

In [ ]:
# === 6. Agregar por estado + cluster ===
df_sumarizado = df_2.groupby(["estado", "cluster"]).agg(
    candidatos=("id_candidato", "count"),
    idade_media=("idade", "mean"),
    escolaridade_media=("nivel_educacional", "mean"),
    perc_com_sap=("experiencia_sap", "mean"),
    remuneracao_media=("remuneracao_mensal_brl", lambda x: x.replace(-9999, np.nan).mean()),
    experiencias_media=("quantidade_experiencias", "mean")
).reset_index()

# Arredondamentos
df_sumarizado["idade_media"] = df_sumarizado["idade_media"].round(1)
df_sumarizado["escolaridade_media"] = df_sumarizado["escolaridade_media"].round(2)
df_sumarizado["perc_com_sap"] = (df_sumarizado["perc_com_sap"] * 100).round(1)
df_sumarizado["remuneracao_media"] = df_sumarizado["remuneracao_media"].round(2)
df_sumarizado["experiencias_media"] = df_sumarizado["experiencias_media"].round(2)

## 04_Salvando_base

In [ ]:
# === 7. Exportar para Parquet e Excel ===
df_sumarizado.to_parquet("03_modelos/05_cluster_estado_resumo.parquet", index=False)
df_sumarizado.to_excel("03_modelos/05_cluster_estado_resumo.xlsx", index=False)

# Exibir preview
df_sumarizado.head()